<a href="https://colab.research.google.com/github/urmzd/lepus-classifier/blob/main/notebooks/example_bootstrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!rm -rf /content/lepus-classifier/
!git clone https://github.com/urmzd/lepus-classifier

Cloning into 'lepus-classifier'...
remote: Enumerating objects: 1139, done.
remote: Counting objects: 100% (1139/1139), done.
remote: Compressing objects: 100% (709/709), done.
remote: Total 1139 (delta 639), reused 749 (delta 330), pack-reused 0
Receiving objects: 100% (1139/1139), 52.73 MiB | 10.03 MiB/s, done.
Resolving deltas: 100% (639/639), done.


In [6]:
!pip install wandb pytorch-lightning pipetools loguru typing_extensions
!pip install pyyaml==5.4.1

In [7]:
!wandb login

wandb: Currently logged in as: csci-4155-rabbit-classifier (use `wandb login --relogin` to force relogin)


In [8]:
import sys
SRC_PATH = "/content/lepus-classifier"
sys.path.insert(0, SRC_PATH)

In [ ]:
from src.bootstrap import bootstrap, TrainerFactory
from src.data.data_handler import BasicModel
from pathlib import Path
from pytorch_lightning.loggers import WandbLogger
import wandb
import pytorch_lightning as pl
import torch
from torchmetrics import Accuracy
import torch.nn.functional as F

trainer_factory = TrainerFactory(strategy=None, max_epochs=5, logger=WandbLogger(project="rabbit-classifier"))

bootstrap(trainer_factory=trainer_factory, data_manfiest_path=Path("/content/lepus-classifier/resources/data.csv"), model=BasicModel())

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loggers/wandb.py:346: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  "There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse"
Global seed set to 42
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type       | Params
---------------------------------------------
0 | layer_1       | Conv2d     | 75    
1 | layer_2       | MaxPool2d  | 0     
2 | layer_3       | ReLU       | 0     
3 | layer_4       | Flatten    | 0     
4 | layer_5       | Linear     | 75.0 K
5 | softmax_layer | LogSoftmax | 0     
---------------------------------------------
75.1 K    Trainable params
0         Non-trainable params
75.1 K    Total pa

Training: 0it [00:00, ?it/s]

/content/lepus-classifier/src/data/data_handler.py:294: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x_5 = self.layer_5(x_4)


Validation: 0it [00:00, ?it/s]